# Biomass burning emissions files perturb and save
Import emissions and calculate monthly sum and average over 2005-2015. Save as new emissions files in loop

In [1]:
import numpy as np
import xarray as xr
import glob
import nc_time_axis
from datetime import datetime

ERROR 1: PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
#make (today's) date label for filename
datelabel = datetime.today().strftime('%Y%m%d')

In [3]:
#define file locations
file_loc1 = '/div/no-backup/users/zofias/CMIP6_bb_ems/' #infile path
out_file = '/div/nac/users/zofias/inputdata/crices_bb_ems/' #path for saving netcdfs

#resolution label
reslab = '0.9x1.25'

#### Select files

In [4]:
bb_files = sorted(glob.glob(f'{file_loc1}*.nc'))
# check filelist
bb_files

['/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_CO_bb_surface_1750-2015_0.9x1.25_c20170322.nc',
 '/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_DMS_bb_surface_1750-2015_0.9x1.25_c20170322.nc',
 '/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_SO2_bb_surface_1750-2015_0.9x1.25_c20170322.nc',
 '/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_SOAGx1.5_bb_surface_1750-2015_0.9x1.25_c20170322.nc',
 '/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_bc_a4_bb_surface_1750-2015_0.9x1.25_c20170322.nc',
 '/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_num_bc_a4_bb_surface_1750-2015_0.9x1.25_c20170322.nc',
 '/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_num_pom_a4_bb_surface_1750-2015_0.9x1.25_c20170509.nc',
 '/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_num_so4_a1_bb_surface_1750-2015_0.9x1.25_c20170322.nc',
 '/div/no-backup/users/zofias/CMIP6_bb_ems/emissions-cmip6_pom_a4_bb_surface_1750-2015_0.9x1.25_c20170322.nc',

In [5]:
# check filename label and adjust if incorrect
# should be e.g. emissions-cmip6_CO_bb_surface
for file in bb_files:
    filelab = file.split('/')[-1][:-32]
    print(filelab)

emissions-cmip6_CO_bb_surface
emissions-cmip6_DMS_bb_surface
emissions-cmip6_SO2_bb_surface
emissions-cmip6_SOAGx1.5_bb_surface
emissions-cmip6_bc_a4_bb_surface
emissions-cmip6_num_bc_a4_bb_surface
emissions-cmip6_num_pom_a4_bb_surface
emissions-cmip6_num_so4_a1_bb_surface
emissions-cmip6_pom_a4_bb_surface
emissions-cmip6_so4_a1_bb_surface


### Make mean and AllBoreal files

In [16]:
for file in bb_files:
    filelab = file.split('/')[-1][:-32]

    #open file and select years 2005-2014
    bb_ems_all = xr.open_dataset(file).sel(time=slice('2005-01-01','2015-01-01'))

    #all vars in dataset
    vars = list(bb_ems_all.keys())
    #var label (second variable in dataset)
    ems_var = vars[1]

    # #select 2010 dates to use in netcdf
    ems_bb_2010 = bb_ems_all.sel(time=slice('2010-01-01','2010-12-30'))

    #Find lat and lon indices
    #where 50N, 60N latitude index is
    lat_ind_50 = np.where(bb_ems_all.lat>=50)[0][0] 

    #calculate sum and average emissions for 2005 - 2015
    emiss_calcs = xr.Dataset() 
    emiss_calcs['bb_average'] = bb_ems_all[ems_var].groupby('time.month').mean(dim='time',keep_attrs=True)
    emiss_calcs['bb_sum'] = bb_ems_all[ems_var].groupby('time.month').sum(dim='time',keep_attrs=True)

    #perturbation file, sum over >50N
    emiss_calcs['bb_ems_pert'] = emiss_calcs['bb_average'].copy()
    emiss_calcs['bb_ems_pert'][:,lat_ind_50:,:] = emiss_calcs['bb_sum'][:,lat_ind_50:,:].values

    #correct time coordinates
    emiss_calcs = emiss_calcs.rename({'month':'time'})
    emiss_calcs = emiss_calcs.assign_coords({'time':ems_bb_2010.time})

    #make copies of dataset for 2010 only to get times and dates correct
    ds_bb_mean = ems_bb_2010.copy(deep=True)
    ds_bb_pert = ems_bb_2010.copy(deep=True)

    #make perturbation file base on mean baseline
    ds_bb_mean[ems_var] = emiss_calcs['bb_average']
    ds_bb_pert[ems_var] = emiss_calcs['bb_ems_pert']

    #save to netcdf
    #ds_bb_mean.to_netcdf(f'{out_file}{filelab}_2010_{reslab}_mean_{datelabel}.nc')
    ds_bb_pert.to_netcdf(f'{out_file}{filelab}_2010_{reslab}_AllBoreal_{datelabel}.nc')

## Smaller regional perturbations

In [8]:
#define regions
#***** change region here to make different files ****
region_label = 'WestBOAS' #'EastBOAS' # 'EastBONA'

regions = {'EastBOAS' : [94,191],
           'WestBOAS' : [50,94],
           'EastBONA' : [224,336]}

In [9]:
for file in bb_files:
    filelab = file.split('/')[-1][:-32]

    #open file and select years 2005-2014
    bb_ems_all = xr.open_dataset(file).sel(time=slice('2005-01-01','2015-01-01'))

    #all vars in dataset
    vars = list(bb_ems_all.keys())
    #var label (second variable in dataset)
    ems_var = vars[1]

    # #select 2010 dates to use in netcdf
    ems_bb_2010 = bb_ems_all.sel(time=slice('2010-01-01','2010-12-30'))

    #Find lat and lon indices
    #where 50N latitude index is
    lat_ind_50 = np.where(bb_ems_all.lat>=50)[0][0] 

    #selects lat lon bounds for region
    lon_min, lon_max = regions[region_label]
    lon_min_ind, lon_max_ind = [np.where(bb_ems_all.lon>=lon_min)[0][0], np.where(bb_ems_all.lon>=lon_max)[0][0]]

    #calculate sum and average emissions for 2005 - 2014
    emiss_calcs = xr.Dataset() 
    emiss_calcs['bb_average'] = bb_ems_all[ems_var].groupby('time.month').mean(dim='time',keep_attrs=True)
    emiss_calcs['bb_sum'] = bb_ems_all[ems_var].groupby('time.month').sum(dim='time',keep_attrs=True)

    #perturbation file, sum over >50N
    emiss_calcs['bb_ems_pert'] = emiss_calcs['bb_average'].copy()
    emiss_calcs['bb_ems_pert'][:,lat_ind_50:,lon_min_ind:lon_max_ind] = emiss_calcs['bb_sum'][:,lat_ind_50:,lon_min_ind:lon_max_ind].values

    #correct time coordinates
    emiss_calcs = emiss_calcs.rename({'month':'time'})
    emiss_calcs = emiss_calcs.assign_coords({'time':ems_bb_2010.time})

    #make copies of dataset for 2010 only to get times and dates correct
    ds_bb_mean = ems_bb_2010.copy(deep=True)
    ds_bb_pert = ems_bb_2010.copy(deep=True)

    #make perturbation file base on mean baseline
    ds_bb_mean[ems_var] = emiss_calcs['bb_average']
    ds_bb_pert[ems_var] = emiss_calcs['bb_ems_pert']

    #save to netcdf
    #ds_bb_mean.to_netcdf(f'{out_file}{filelab}_2010_{reslab}_mean_{datelabel}.nc')
    ds_bb_pert.to_netcdf(f'{out_file}{filelab}_2010_{reslab}_{region_label}_{datelabel}.nc')